# Modularity calculation
Calculating the modularity of a higher order network may take a long time - especially when looping over all the pairs of nodes. Observe, that for a network $n$ first-order nodes (each with constant degrees $d$), the number of $k$-th order nodes is $n\cdot d^{order-1}$. Therefore, the adjacency matrix is very sparse and its number of non-zero entries is only $n\cdot d^{order}$. Hence, it is crucial to avoid iterating over all $n^2\cdot d^{2order-2}$ pairs of nodes.

The modularity $q$ compares the number of actual edges connecting nodes of the same class with its expected counterpart.
To count the actual edges, we need only to iterate over the non-zero entries of the adjacency matrix.
While the term containing the expected number of edges apparently needs a loop over all pairs of nodes, its special structure (outer product) enables a faster calculation:
$\sum_{i,j} a[i]\cdot b[j]\cdot 1_{C[i]=C[j]} = \sum_c (\sum_i a[i]\cdot 1_{C[i]=c}) (\sum_j b[j]\cdot 1_{C[j]=c})$

Furthermore, the implementation supports generalization of modularity to 
* directed networks, as in Leicht and Newman (2007), "Community structure in directed networks"
* weighted networks, as in Newman (2004), "Analysis of weighted networks"

In [1]:
# this workbook needs pathpy2 installed
%conda list pathpy

# packages in environment at C:\Users\mstud\Anaconda3\envs\pathpy2:
#
# Name                    Version                   Build  Channel
pathpy2                   2.2.0                    pypi_0    pypi
Note: you may need to restart the kernel to use updated packages.



In [2]:
import pathpy as pp
import numpy as np
from collections import defaultdict

In [3]:
def calc_q(net, C, weighted=False, **kwargs):
    nodes = list(net.nodes.keys())
    mat = net.adjacency_matrix(weighted=weighted, **kwargs) # scipy.sparse.csc.csc_matrix
    mat_sum = 0
    mat_sum_by_row_C = defaultdict(float)
    mat_sum_by_col_C = defaultdict(float)
    q = 0
    for c in range(mat.shape[1]):
        c_C = C[nodes[c]] # class of current column's node
        for ind in range(mat.indptr[c], mat.indptr[c+1]):
            r = mat.indices[ind]
            v = mat.data[ind]
            # assert mat[r,c]==v
            r_C = C[nodes[r]] # class of current rows's node
            mat_sum += v
            mat_sum_by_row_C[r_C] += v
            mat_sum_by_col_C[c_C] += v
            if c_C == r_C:
                q += v
    q_exp = sum( v*mat_sum_by_col_C[c] for c,v in mat_sum_by_row_C.items() ) / (mat_sum**2)
    q = q/mat_sum - q_exp
    q_max = 1 - q_exp
    return (q, q_max)

calc_q() still uses for loops. Consider converting with .tocoo(), which represents the matrix with three arrays (row, col, data), and use numpy array operations.

## Some testdata
to compare with existing implementation

In [4]:
n = 100
net = pp.algorithms.random_graphs.erdoes_renyi_gnp(n, 0.2, self_loops=False, directed=False)
for e in net.edges:
    net.edges[e]['weight'] = np.random.exponential()
net_D = pp.algorithms.random_graphs.erdoes_renyi_gnp(n, 0.2, self_loops=False, directed=True)
for e in net_D.edges:
    net_D.edges[e]['weight'] = np.random.exponential()

In [5]:
classes = { str(i): 'group %d' % (i%5) for i in range(n) }

In [6]:
q = pp.algorithms.modularity.q(net, C=classes)
q_max = pp.algorithms.modularity.q_max(net, C=classes)
if q_max < 0:
    q_max+= 1
print(q,q_max)

-0.016743908659001307 0.7998893578740652


In [7]:
calc_q(net, classes, transposed=True) # weighted=False

(-0.016743908659001383, 0.7998893578740648)

Calculation works for all variants of adjacency matrices (weighted and/or directed):

In [8]:
calc_q(net, classes, weighted=True, transposed=True)

(-0.01852957868856711, 0.798714644094694)

In [9]:
calc_q(net_D, classes, weighted=True, transposed=True)

(0.010614981011942137, 0.8010128521279705)

In [10]:
# calculation is invariant under transposition of the matrix
calc_q(net_D, classes, weighted=True, transposed=False)

(0.010614981011942137, 0.8010128521279705)

In [11]:
# works also for HigherOrderNetwork.adjacency_matrix(include_subpaths, weighted, transposed)
# calc_q(hon, classes, weighted=True, include_subpaths=True)